In [7]:
import nltk
from collections import Counter
import glob
import os
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Paths for all the different corpora
folderpaths = ['../Corpora/AnxietyMeditations/', '../Corpora/SleepMeditations/',
               '../Corpora/LearningAndGrowthMeditations/', '../Corpora/MorningMeditations/']

#array to store the text of each file 
corpus = []
text2 = ""

for folder in folderpaths:
    if folder == '../Corpora/AnxietyMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text
    elif folder == '../Corpora/SleepMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text
    elif folder == '../Corpora/LearningAndGrowthMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text
    elif folder == '../Corpora/MorningMeditations/':
        for doc in glob.glob(os.path.join(folder, '*.txt')):
            with open(doc, 'r') as f:
                text = f.read()
                corpus.append(text)
                text2 += text

newStopWords = ['[', ']', '`', '#', '$',
                '(', ')', "Music", ':', "....", "--", ",", "''", '`', '"']
stopwords = set(stopwords.words('english') +
                newStopWords + list(string.punctuation))

# Tokenizing by word and filtering stopwords
all_tokens = [w for w in word_tokenize(text2.lower()) if not w in stopwords]

In [8]:
import nltk
from nltk.tokenize import word_tokenize

# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    # Tokenize and lowercase
    tokens = [w for w in word_tokenize(text.lower()) if not w in stopwords]
    tokenized_corpus.append(tokens)


In [9]:
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
# Preprocess the text data
tokenized_corpus = []
for text in corpus:
    tokens = word_tokenize(text.lower())  # Tokenize and lowercase
    tokenized_corpus.append(tokens)

context_windows = []
window_size = 4  # Size of the context window

for sentence in tokenized_corpus:
    sentence_length = len(sentence)
    for i in range(sentence_length):
        # Extract the target word
        target_word = sentence[i]

        # Extract the context words within the window size
        context_words = []
        for j in range(max(0, i - window_size), min(sentence_length, i + window_size + 1)):
            if j != i:
                context_words.append(sentence[j])

        # Append the target word and its context words as a tuple
        # Convert context_words to tuple
        context_windows.append((target_word, tuple(context_words)))

# Train the Word2Vec model
model = Word2Vec(context_windows, window=5,
                 min_count=10, sg=0)  # CBOW approach

# Accessing trained word vectors
word_vector = model.wv['breathe']  # Get the word vector for a specific word
print(word_vector)

[-0.04611077  0.05143551  0.00832617  0.10568035  0.0678573  -0.05113725
  0.08877496  0.05135308 -0.0924836   0.00625963 -0.00269381 -0.10847999
  0.01868773 -0.0461509   0.01276269  0.04809852 -0.01082935 -0.08588783
 -0.01512117 -0.22306041 -0.00630978  0.09596051  0.10741212 -0.06557321
 -0.01084703  0.06969886 -0.04669424 -0.00948677 -0.12056852  0.00332942
  0.09238226  0.03065117  0.0625176  -0.01604205 -0.10206982  0.09264807
 -0.00654783  0.00070246 -0.05666429 -0.12213346  0.09708432 -0.02240049
 -0.02771159 -0.06025636  0.06934972 -0.04438044 -0.06740531  0.07275707
  0.0979266   0.03026421 -0.01174417 -0.01931076  0.03403663  0.02161161
 -0.05970366  0.04387289  0.00672422 -0.07817086 -0.01326801 -0.03930199
  0.00552479  0.05517378  0.09445991  0.01742239 -0.04255137  0.0171051
 -0.11576808  0.02523129 -0.09439326 -0.01464783 -0.00791736  0.09378819
  0.02240686 -0.02210553  0.02032431 -0.02899616  0.02707856 -0.09984975
 -0.03723887 -0.0573241  -0.00680383 -0.06865994 -0.

In [10]:
import os
import nltk
from gensim.models import Word2Vec

# Define the directory containing the text files
directory = "./txt Files/"

all_vectors = { 'sight': [], 'smell': [], 'sound': [], 'taste': []
, 'touch': []}

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(directory, filename)
        with open(filepath, "r") as f:
            text = f.read()
            words = nltk.word_tokenize(text)
            for word in words:
                try:
                    word_vector = model.wv[word.lower()]
                    all_vectors[filename[:-4]].append(word_vector)
                    # print(f"Word: {word}\nVector: {word_vector}\n")
                except KeyError:
                    continue
                    # print(f"Word '{word}' not in vocabulary.\n")

# print(len(all_vectors['sight']))
# print(len(all_vectors['sight']))


In [12]:
import numpy as np
from scipy.spatial.distance import cosine

for key in all_vectors.keys(): 
    # Calculate the pairwise cosine similarity distance between each row in the matrix
    sight_len = len(all_vectors[key])
    distances = np.zeros((sight_len, sight_len))
    for i in range(sight_len):
        for j in range(i+1, sight_len):
            distances[i][j] = cosine(all_vectors[key][i], all_vectors[key][j])
            distances[j][i] = distances[i][j]

    # Print the resulting distance matrix
    distances = distances[:10, :10]
    print(distances.shape)

# Normalize the matrix to the range 0-1
# normalized_distance = (distances - np.min(distances)) / (np.max(distances) - np.min(distances))

(10, 10)
(10, 10)
(10, 10)
(10, 10)
(10, 10)
